# Load packages

In [1]:
!pip install pandas
!pip install openpyxl

import glob
import os

import pandas as pd

# Load the sample list

The list of samples selected for the proxy-proxy calibration needs to have certain format in order to be compatible with this code.
See the file structure below. 

In [2]:
df_lookup = pd.read_excel('../data/SourceData/Calib_samples.xlsx')
print(df_lookup)

   SampleName  DepthStart_mm  DepthEnd_mm  weight (g)  dilution  Total volume
0        C1-1            510          520      0.5066         1             3
1        C1-2            570          580      0.5265         1             3
2        B1-2            300          310      0.5293         1             3
3        B1-2            400          410      0.5072         1             3
4        B1-2            500          510      0.5070         1             3
5        B1-2            800          810      0.5112         1             3
6        B1-2            900          910      0.5026         1             3
7        A2-2            700          710      0.5028         1             5
8        A2-2            900          910      0.5180         1             5
9        A2-3            300          310      0.5123         1            10
10       A2-3            400          410      0.5094         1            10
11       A2-3            900          910      0.5250         1 

# Load the folder with HSI data

All the hyperspectral data should be placed in one HSI folder. Each core, which was subsampled from proxy-proxy proxy calibation needs to be in a separate csv file (output of e.g., napari-sediment) and the file needs to contain the SampleName from the list of samples.
Example: core C1-1 subsample will be looked up in a file which has C1-1 or C1_1 in its name: "C1_1_index_projection.csv".

### Looks up all the csv files in the folder

In [3]:
path = r'../data/HSI'
all_files = glob.glob(os.path.join(path, "*.csv"))
print(all_files)

['../data/HSI/C1_2_index_projection.csv', '../data/HSI/C1_1_index_projection.csv', '../data/HSI/A2_2_index_projection.csv', '../data/HSI/B1_2_index_projection.csv', '../data/HSI/A2_3_index_projection.csv']


### Extraction of the dataframe names from the csv files

In [4]:
df_index = {}

for f in all_files:
    sample_name = os.path.basename(f)[:4].replace('_', '-')  # sample_name == 'B1-2' etc., this may be changed based on the file naming.
    print(sample_name)
    sample_df = pd.read_csv(f)
    # display(sample_df)
    df_index[sample_name] = sample_df

C1-2
C1-1
A2-2
B1-2
A2-3


# Extracting the average of RABD data in the given intervals from the sample list

In [5]:
depth_col = 'depth [mm]' #name of the depth column in the HSI data used for looking up the ranges of calibration samples

for index, row in df_lookup.iterrows():
    sample_name = row['SampleName']
    start = row['DepthStart_mm']
    end = row['DepthEnd_mm']
    df = df_index[sample_name]
    df = df[(df[depth_col] >= start) & (df[depth_col] <= end)]
    mean_values = df.filter(like='RABD', axis=1).mean()
    for col, val in mean_values.items():
        df_lookup.at[index, col] = val
display(df_lookup)

,SampleName,DepthStart_mm,DepthEnd_mm,weight (g),dilution,Total volume,RABD620,RABD670,RABD715,RABD845,RABD830
0,C1-1,510,520,0.5066,1,3,1.000543,0.999554,1.002725,0.998265,0.991278
1,C1-2,570,580,0.5265,1,3,1.000758,1.003030,1.003396,0.997679,0.991728
2,B1-2,300,310,0.5293,1,3,1.000230,0.997570,1.006509,0.999043,0.993018
3,B1-2,400,410,0.5072,1,3,1.001288,0.995181,1.003832,1.000731,1.000873
4,B1-2,500,510,0.5070,1,3,1.002085,0.996406,1.005268,0.999786,1.000257
5,B1-2,800,810,0.5112,1,3,1.001102,0.999079,1.003776,0.998330,0.992447
6,B1-2,900,910,0.5026,1,3,1.002708,0.996101,1.005398,0.999266,1.002791
7,A2-2,700,710,0.5028,1,5,1.003503,1.012935,0.996508,1.000880,0.991702
8,A2-2,900,910,0.5180,1,5,1.002802,1.012306,0.996479,0.999743,0.992237
9,A2-3,300,310,0.5123,1,10,1.008281,1.038394,1.068685,1.005900,1.003254


# Saving the updated dataframe

In [ ]:
df_lookup.to_excel("../data/Calib_RABDs.xlsx")

: 